# cuML

我们要探索的另一个包是 [cuML](https://github.com/rapidsai/cuml)。与 `scikit-learn` 类似，您可以使用 `cuml` 在数据上训练机器学习模型来进行预测。与 RAPIDS 工具套件中的其他包一样，`cuml` 的 API 与 `scikit-learn` 相同，但底层代码已经实现为在 GPU 上运行。

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
import cudf

让我们来看看训练一个 K 近邻模型，根据一些其他属性（如血压、血糖水平、BMI 等）来预测一个人是否患有糖尿病。

我们首先使用 `cudf` 将数据加载到 GPU 数据框中。

In [ ]:
df = cudf.read_csv("https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-python-tutorial/data/diabetes.csv")
df.head()

接下来，我们需要创建两个独立的表。一个包含除糖尿病列之外的患者属性，另一个只包含糖尿病列。

In [ ]:
X = df.drop(columns=["Outcome"])
X.head()

In [ ]:
y = df["Outcome"].values
y[0:5]

接下来，我们需要使用 `cuml` 中的 `train_test_split` 方法将数据分成两组。

第一个较大的组将用于训练我们的模型。我们将从每个表中取 80% 的数据，称为 `X_train` 和 `y_train`。在训练模型时，它将能够看到这两组数据以执行聚类。

剩余 20% 的数据将被称为 `X_test` 和 `y_test`。一旦我们的模型训练完成，我们将把 `X_test` 数据输入到模型中，预测这些人是否患有糖尿病。然后我们可以将这些预测与实际的 `y_test` 数据进行比较，看看我们的模型有多准确。

我们还设置了 `random_state=1` 以使随机选择保持一致，这仅用于本教程的目的。我们还设置了 `stratify`，这意味着如果我们初始数据中 75% 的人患有糖尿病，那么我们的训练集中也将保证有 75% 的人患有糖尿病。

In [ ]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

现在我们有了训练数据，可以从 `cuml` 导入 `KNeighborsClassifier` 并拟合我们的模型。

In [ ]:
from cuml.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(X_train,y_train)

我们的模型在 GPU 上完成了拟合，现在我们可以进行一些预测了。让我们预测测试集中的前五个人。

In [ ]:
knn.predict(X_test)[0:5]

我们可以看到，我们的新模型认为第一个患者患有糖尿病，但其他人没有。

让我们将整个测试集和实际答案一起通过评分函数运行，看看我们的模型表现如何。

In [ ]:
knn.score(X_test, y_test)

恭喜！您刚刚在 Python 中使用 GPU 训练了一个机器学习模型，并获得了 69% 的准确率。这里有很多方法可以提高这个分数，但这超出了本教程的范围。